# FL Vanilla Mode using syft





**Author** 
: Laveena Kewlani

**Objective** : To run a decentralized model uisng MNIST dataset on FL vanilla architecture

In [2]:
# Reference : https://pypi.org/project/syft/
#
!pip install syft

In [ ]:
import torch as th
import numpy as np
from torchvision import datasets, transforms
import torchvision.datasets as datasets
from torch.utils.data import Subset
from torch import nn
import torch.nn.functional as F
from torch import optim
import syft as sy
import helper

#Hooking syft to torch
hook = sy.TorchHook(th)

In [1]:
#Method to create 10 virtual workers and move to a list of workers
def create_workers():
  workers = []
  cartman = sy.VirtualWorker(hook, id = "cartman")
  workers.append(cartman)
  kyle = sy.VirtualWorker(hook, id = "kyle")
  workers.append(kyle)
  kenny = sy.VirtualWorker(hook, id = "kenny")
  workers.append(kenny)
  stan = sy.VirtualWorker(hook, id = "stan")
  workers.append(stan)
  butters = sy.VirtualWorker(hook, id = "butters")
  workers.append(butters)
  wendy = sy.VirtualWorker(hook, id = "wendy")
  workers.append(wendy)
  heidi = sy.VirtualWorker(hook, id = "heidi")
  workers.append(heidi)
  bebe = sy.VirtualWorker(hook, id = "bebe")
  workers.append(bebe)
  nichole = sy.VirtualWorker(hook, id = "nichole")
  workers.append(nichole)
  patty = sy.VirtualWorker(hook, id = "patty")
  workers.append(patty)
  return workers


In [2]:
#Method to clear out every tensor stored in the list of virtual workers
def clear_workers(workers):
  for worker in workers:
    worker.clear_objects()

In [3]:
#Method to split the mnist test dataset into the various workers and also to load the mnist test dataset into a test loader
def create_federated_and_test_loaders(workers, trainset, testset):
  federated_train_loader = sy.FederatedDataLoader(
      trainset.federate(workers), 
      batch_size=32, shuffle=True)

  test_loader = th.utils.data.DataLoader(testset, batch_size=64, shuffle=False)
  return federated_train_loader, test_loader

In [4]:
#Method to train models on the virtual workers without moving any gradient to the central models until the gradients have been coallated.
def create_train_federated_models(workers, loader, lr = 0.12, epoch = 5):
  #sends model to first virtual worker
  virtual_model = classifier().send(workers[0])
  optimizer = optim.SGD(virtual_model.parameters(), lr)
  criterion = nn.NLLLoss()
  for n in range(epoch):
    
    #Integer to keep up with first index.
    i = 0
    
    #Integer to keep up with current worker while training
    j = 0
    
    #Integer to count number of mini-batches per worker
    n_mbatch = 0
    
    #Variable to keep up with current worker while looping
    dbLoc = None
    
    #Variable to store cummulative loss.
    cum_loss = 0
    
    
    for batch_idx, (imgs, labels) in enumerate(loader):
      
      #condition to set dbLoc to the first worker
      if i == 0:
        i = 2
        dbLoc = imgs.location
        
      #condition to change dbLoc if img is stored on a different worker and also calculate loss
      if dbLoc is not imgs.location:
        print("The total loss for {0} for epoch {2} is {1}".format(workers[j].id, cum_loss / n_mbatch, n+1))
        dbLoc = imgs.location
        j += 1
        
        #Moving the model to a new worker
        virtual_model.move(dbLoc)
        
        #Resetting the cummulative loss and batch count to zero for new worker
        cum_loss = 0
        n_mbatch = 0

      optimizer.zero_grad()
      output = virtual_model.forward(imgs)
      loss = criterion(output, labels)
      loss.backward()
      optimizer.step()
      cum_loss +=  loss.get().item()
      n_mbatch += 1
    print("The total loss for {0} is {1}".format(workers[j].id, cum_loss / n_mbatch))
    
    #Moving the model to the first worker if training would occur again
    if (n < (epoch - 1)):
      virtual_model.move(workers[0])
  return virtual_model
  
    
  

In [5]:
#Method to return the model to the central machine
def create_central_model(model):
  return model.get()

In [6]:
#Method to analyze the private database with the trained model
def analyze_model(model, loader):
  print("Running on ", "cpu")
  cum_perc = 0
  for imgs, labels in loader:
    with th.no_grad():
      ps =  th.exp(model.forward(imgs))
    top_p, top_class = ps.topk(1, dim = 1)
    prob = top_class == labels.view(*top_class.shape)
    prob = prob.float()
    cum_perc += prob.mean().float()
  print("The accuracy of the model is {0}%".format((cum_perc / len(loader)) * 100))  

In [7]:
#Classifier for creating the models
class classifier(nn.Module):
  def __init__(self):
    super().__init__() 
    self.fc1 = nn.Linear(784, 256)
    self.fc2 = nn.Linear(256, 128)
    self.fc3 = nn.Linear(128, 64)
    self.fc4 = nn.Linear(64, 32)
    self.fc5 = nn.Linear(32, 10)
    
  def forward(self, x):
    x = x.view(x.shape[0], -1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = F.relu(self.fc4(x))
    x = F.log_softmax(self.fc5(x), dim = 1)   
    return x
 

NameError: name 'nn' is not defined

In [8]:
# Application of transforms to normalize the mnist data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

NameError: name 'transforms' is not defined

In [0]:
workers = create_workers()
clear_workers(workers)
federated_loader, test_loader = create_federated_and_test_loaders(workers, mnist_trainset, mnist_testset)


In [27]:
virtual_model = create_train_federated_models(workers, federated_loader, epoch = 2)

The total loss for cartman for epoch 1 is 1.7833157254660383
The total loss for kyle for epoch 1 is 0.9024252550716095
The total loss for kenny for epoch 1 is 0.6204849152647435
The total loss for stan for epoch 1 is 0.4085380588757231
The total loss for butters for epoch 1 is 0.3886932226571631
The total loss for wendy for epoch 1 is 0.35908689480671224
The total loss for heidi for epoch 1 is 0.3309765687172717
The total loss for bebe for epoch 1 is 0.29516828555534497
The total loss for nichole for epoch 1 is 0.2855488977375183
The total loss for patty is 0.174475033589183
The total loss for cartman for epoch 2 is 0.22575426379099806
The total loss for kyle for epoch 2 is 0.20347901956832154
The total loss for kenny for epoch 2 is 0.2153895103392449
The total loss for stan for epoch 2 is 0.17871475322766506
The total loss for butters for epoch 2 is 0.18308901988921014
The total loss for wendy for epoch 2 is 0.18084254954010248
The total loss for heidi for epoch 2 is 0.174303403084582

In [0]:
central_model = create_central_model(virtual_model)

In [29]:
analyze_model(central_model, test_loader)

Running on  cpu
The accuracy of the model is 96.1783447265625%
